# Quick example usage

The main tool in fractrics is the MSM class, an implementation of the univariate [Markov Switching Multifractal Model](https://en.wikipedia.org/wiki/Markov_switching_multifractal). The logaritmic difference between observations is modeled as the noise-adjusted square root of the product of a chosen number of latent volatility components, each following the dynamics of discrete first order markov chains, whose transition depends on geometrically-spaced Poisson arrivals, and an unconditional term, effectively being the unconditional volatility.

Such structure effectively captures the behaviour of time series with fat tails, hyperbolic correlation decay, and multifractal moments, such as the returns of many financial assets.

The implementation is made in JAX, simplifying parallelization of the code. Moreover, following from [this](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=5276090) paper, the memory complexity of the forward algorithm is reduced, due to the factorization of latent states.

To use the model, start with an example time series. Note that the model is only defined for positive time series (as it was created to model prices of financial assets).

In [1]:
from fractrics.time_series.MSM import MSM
import jax.numpy as jnp
import numpy as np

ts_test = jnp.array(np.random.normal(50, 10, 10))

Then initialize the model. It requires the following hyperparameters:
 - `num_latent`: how many volatility components, integer.
 - `marg_prob_mass`: the probability mass of the marginal distribution of the latent states, needs to sum to 1. 

In [2]:
model = MSM(ts=ts_test, num_latent=3)

To fit the model to the data, start with an initial guess. The `MSM.fit()` method then optimizes the parameters using `jaxopt`'s [Broyden–Fletcher–Goldfarb–Shanno algorithm](https://en.wikipedia.org/wiki/Broyden%E2%80%93Fletcher%E2%80%93Goldfarb%E2%80%93Shanno_algorithm).

By assumption, all the parameters need to be positive, and have further individual constrains:

- `marg_support`: the support of the marginal probability mass defined in the parameters. It needs to have unity expectation. In the symmetric binomial case, this can be enforced by specifying one value $m_0$, and having the second value be $2 - m_0$.

- `unconditional_term`: the unconditional distribution of the model, a positive double.

- `arrival_gdistance`: the geometric distance between the Poisson arrivals of each latent volatility component, a positive double.

- `hf_arrival`: the highest poisson arrival probability (i.e. the proability of state switch of the highest frequency component).

Note: to maintain the constrains during optimization, the parameters are transformed using mappings.

In [4]:
initial_params = jnp.array([
    2,    #unconditional term
    3.0,    #arrival_gdistance
    0.98,   #hf_arrival
    #support
    1.5,    
    0.5
])
fitresult = model.fit(initial_parameters=initial_params, maxiter=1000, verbose=False)

params, current_distribution, transition_tensor, distribution_history, negative_log_likelihood = fitresult
print(params)


[0.22855803 3.1763792  0.44004682 1.0000163  0.9999837 ]


It is also possible to make simulations with the MSM. To avoid tracing issues with JAX, the parameters are to be given as input for the simulation. Follows an example with the parameters of the fitted model above.

In [5]:
num_latent = model.num_latent
unconditional_term = params[0]
arrival_gdistance = params[1]
hf_arrival = params[2]
marg_support = params[3:]

poisson_arrivals = 1 - (1 - hf_arrival) ** (1 / (arrival_gdistance ** (jnp.arange(num_latent, 0, -1) - 1)))

simulation = model.simulation(number_simulations= 1000, unconditional_term = unconditional_term,
                              poisson_arrivals=poisson_arrivals,
                              marginal_support = marg_support
)


Finally a 7 period forecast. The transition tensor and current distribution are required as input

In [6]:
forecast = model.forecast(7, current_distribution, *transition_tensor)